# 1단계: 네트워크 인프라 구성

### 1.1 On-premise VNET 생성

**Azure Portal:**

1.  Azure Portal에 로그인합니다.
2.  "Create a resource"를 클릭합니다. (예시 스크린샷: "+" 버튼 클릭)
3.  "Virtual network"을 검색하여 선택하고 "Create"를 클릭합니다. (예시 스크린샷: 검색창에 "Virtual network" 입력 후 선택)
4.  다음 정보를 입력하고 "Review + create"를 클릭합니다.
    -   Subscription: 사용 중인 Azure 구독을 선택합니다.
    -   Resource group: "secu-ai-workshop"을 선택합니다.
    -   Name: "onprem-vnet"
    -   Region: "Korea Central"
    -   IP Addresses 탭에서 Address space를 "192.168.0.0/16"으로 설정합니다.
    -   Subnets 탭에서 다음 서브넷을 추가합니다.
        -   onprem-subnet: 192.168.1.0/24
        -   GatewaySubnet: 192.168.0.0/24 (주의: GatewaySubnet 이름은 변경 불가)
5.  "Create"를 클릭하여 VNET 생성을 완료합니다.

In [ ]:
# Azure CLI 로그인
# 이 스크립트는 Azure CLI에 로그인하는 데 사용됩니다.
# 이 명령어는 터미널에서 직접 수행하고 그 다음부터는 shell에서 Azure CLI를 사용할 수 있습니다.
# 터미널에서 수행할 때에는 !없이 수행합니다.

!az login

In [ ]:
# Azure CLI 버전 확인
# azure-cli 2.71.0 이상의 버전을 사용할 것을 권장합니다.
!az version

{
  "azure-cli": "2.71.0",
  "azure-cli-core": "2.71.0",
  "azure-cli-telemetry": "1.1.0",
  "extensions": {
    "azure-firewall": "1.2.2",
    "bastion": "1.4.0"
  }
}


In [2]:
# 리소스 그룹 생성
RESOURCE_GROUP="secu-ai-workshop"
LOCATION="koreacentral"

In [ ]:
# 리소스 그룹 생성

!az group create --name $RESOURCE_GROUP --location $LOCATION

{
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop",
  "location": "koreacentral",
  "managedBy": null,
  "name": "secu-ai-workshop",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [3]:
# 변수 설정
ONPREM_VNET_NAME="onprem-vnet"
ONPREM_ADDRESS_PREFIX="192.168.0.0/16"
ONPREM_SUBNET_NAME="onprem-subnet"
ONPREM_SUBNET_PREFIX="192.168.1.0/24"
GATEWAY_SUBNET_NAME="GatewaySubnet" # GatewaySubnet 이름은 고정
GATEWAY_SUBNET_PREFIX="192.168.0.0/24"

In [4]:
# On-premise VNET 생성
!az network vnet create \
    --name $ONPREM_VNET_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --address-prefixes $ONPREM_ADDRESS_PREFIX \
    --subnet-name $ONPREM_SUBNET_NAME \
    --subnet-prefixes $ONPREM_SUBNET_PREFIX

# GatewaySubnet 생성
!az network vnet subnet create \
    --name $GATEWAY_SUBNET_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $ONPREM_VNET_NAME \
    --address-prefix $GATEWAY_SUBNET_PREFIX

{
  "newVNet": {
    "addressSpace": {
      "addressPrefixes": [
        "192.168.0.0/16"
      ]
    },
    "enableDdosProtection": false,
    "etag": "W/\"0949c8c5-e833-47c5-aa97-42f25af91790\"",
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/onprem-vnet",
    "location": "koreacentral",
    "name": "onprem-vnet",
    "privateEndpointVNetPolicies": "Disabled",
    "provisioningState": "Succeeded",
    "resourceGroup": "secu-ai-workshop",
    "resourceGuid": "0675f17b-a885-406d-b930-5afc6a52d290",
    "subnets": [
      {
        "addressPrefix": "192.168.1.0/24",
        "delegations": [],
        "etag": "W/\"0949c8c5-e833-47c5-aa97-42f25af91790\"",
        "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/onprem-vnet/subnets/onprem-subnet",
        "name": "onprem-subnet",
        "privateEndpointNetworkPol

#### 1.2 Hub VNET 생성

**Azure Portal:**

1.  Azure Portal에서 "Create a resource"를 클릭합니다.
2.  "Virtual network"을 검색하여 선택하고 "Create"를 클릭합니다.
3.  다음 정보를 입력하고 "Review + create"를 클릭합니다.
    -   Subscription: 사용 중인 Azure 구독을 선택합니다.
    -   Resource group: "secu-ai-workshop"을 선택합니다.
    -   Name: "hubvnet"
    -   Region: "Korea Central"
    -   IP Addresses 탭에서 Address space를 "10.0.0.0/16"으로 설정합니다.
    -   Subnets 탭에서 다음 서브넷을 추가합니다.
        -   GatewaySubnet: 10.0.0.0/24
        -   AzureFirewallSubnet: 10.0.1.0/26 (주의: AzureFirewallSubnet 이름은 변경 불가)
        -   dmz: 10.0.2.0/24
4.  "Create"를 클릭하여 VNET 생성을 완료합니다.

In [5]:
# 변수 설정
HUB_VNET_NAME="hubvnet"
HUB_ADDRESS_PREFIX="10.0.0.0/16"
HUB_GATEWAY_SUBNET_PREFIX="10.0.0.0/24"
FIREWALL_SUBNET_NAME="AzureFirewallSubnet" # AzureFirewallSubnet 이름은 고정
FIREWALL_SUBNET_PREFIX="10.0.1.0/26"
DMZ_SUBNET_NAME="dmz"
DMZ_SUBNET_PREFIX="10.0.2.0/24"

In [6]:
# Hub VNET 생성
!az network vnet create \
    --name $HUB_VNET_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --address-prefixes $HUB_ADDRESS_PREFIX \
    --subnet-name $GATEWAY_SUBNET_NAME \
    --subnet-prefixes $HUB_GATEWAY_SUBNET_PREFIX

# AzureFirewallSubnet 생성
!az network vnet subnet create \
    --name $FIREWALL_SUBNET_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $HUB_VNET_NAME \
    --address-prefix $FIREWALL_SUBNET_PREFIX

# dmz 서브넷 생성
!az network vnet subnet create \
    --name $DMZ_SUBNET_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $HUB_VNET_NAME \
    --address-prefix $DMZ_SUBNET_PREFIX

{
  "newVNet": {
    "addressSpace": {
      "addressPrefixes": [
        "10.0.0.0/16"
      ]
    },
    "enableDdosProtection": false,
    "etag": "W/\"9c176a94-178b-4534-9ac5-a10e8a491bbd\"",
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/hubvnet",
    "location": "koreacentral",
    "name": "hubvnet",
    "privateEndpointVNetPolicies": "Disabled",
    "provisioningState": "Succeeded",
    "resourceGroup": "secu-ai-workshop",
    "resourceGuid": "61a46c7a-007d-46a8-a06d-01e8f34643be",
    "subnets": [
      {
        "addressPrefix": "10.0.0.0/24",
        "delegations": [],
        "etag": "W/\"9c176a94-178b-4534-9ac5-a10e8a491bbd\"",
        "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/hubvnet/subnets/GatewaySubnet",
        "name": "GatewaySubnet",
        "privateEndpointNetworkPolicies": "Disabled"

#### 1.3 Spoke1 VNET 생성

**Azure Portal:**

(Hub VNET 생성 과정과 유사하며, Subnet 구성만 다름)

In [7]:
# 변수 설정
SPOKE1_VNET_NAME="spoke1vnet"
SPOKE1_ADDRESS_PREFIX="10.1.0.0/16"
SPOKE1_SUBNET_NAME="spoke1subnet"
SPOKE1_SUBNET_PREFIX="10.1.0.0/24"
SPOKE1_PE_SUBNET_NAME="privateendpoint"
SPOKE1_PE_SUBNET_PREFIX="10.1.1.0/24"

In [8]:
# Spoke1 VNET 생성
!az network vnet create \
  --name $SPOKE1_VNET_NAME \
  --resource-group $RESOURCE_GROUP \
  --location $LOCATION \
  --address-prefixes $SPOKE1_ADDRESS_PREFIX \
  --subnet-name $SPOKE1_SUBNET_NAME \
  --subnet-prefixes $SPOKE1_SUBNET_PREFIX

# Spoke1 privateendpoint 서브넷 생성
!az network vnet subnet create \
  --name $SPOKE1_PE_SUBNET_NAME \
  --resource-group $RESOURCE_GROUP \
  --vnet-name $SPOKE1_VNET_NAME \
  --address-prefix $SPOKE1_PE_SUBNET_PREFIX

{
  "newVNet": {
    "addressSpace": {
      "addressPrefixes": [
        "10.1.0.0/16"
      ]
    },
    "enableDdosProtection": false,
    "etag": "W/\"f0964cbf-b5e1-4ebc-bd14-f13cad02b9d0\"",
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke1vnet",
    "location": "koreacentral",
    "name": "spoke1vnet",
    "privateEndpointVNetPolicies": "Disabled",
    "provisioningState": "Succeeded",
    "resourceGroup": "secu-ai-workshop",
    "resourceGuid": "6c2ead40-3af9-4626-a264-f52c5dd1be88",
    "subnets": [
      {
        "addressPrefix": "10.1.0.0/24",
        "delegations": [],
        "etag": "W/\"f0964cbf-b5e1-4ebc-bd14-f13cad02b9d0\"",
        "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke1vnet/subnets/spoke1subnet",
        "name": "spoke1subnet",
        "privateEndpointNetworkPolicies": "Di

#### 1.4 Spoke2 VNET 생성

**Azure Portal:**

(Hub VNET 생성 과정과 유사하며, Subnet 구성만 다름)

In [9]:
# 변수 설정
SPOKE2_VNET_NAME="spoke2vnet"
SPOKE2_ADDRESS_PREFIX="10.2.0.0/16"
SPOKE2_SUBNET_NAME="spoke2subnet"
SPOKE2_SUBNET_PREFIX="10.2.0.0/24"

In [10]:
# Spoke2 VNET 생성
!az network vnet create \
  --name $SPOKE2_VNET_NAME \
  --resource-group $RESOURCE_GROUP \
  --location $LOCATION \
  --address-prefixes $SPOKE2_ADDRESS_PREFIX \
  --subnet-name $SPOKE2_SUBNET_NAME \
  --subnet-prefixes $SPOKE2_SUBNET_PREFIX

{
  "newVNet": {
    "addressSpace": {
      "addressPrefixes": [
        "10.2.0.0/16"
      ]
    },
    "enableDdosProtection": false,
    "etag": "W/\"bf7943f7-1e28-4c60-9f6a-d6fdbb4e0863\"",
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke2vnet",
    "location": "koreacentral",
    "name": "spoke2vnet",
    "privateEndpointVNetPolicies": "Disabled",
    "provisioningState": "Succeeded",
    "resourceGroup": "secu-ai-workshop",
    "resourceGuid": "bc8cd8d4-d007-4c2c-8814-ab452c2acad3",
    "subnets": [
      {
        "addressPrefix": "10.2.0.0/24",
        "delegations": [],
        "etag": "W/\"bf7943f7-1e28-4c60-9f6a-d6fdbb4e0863\"",
        "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke2vnet/subnets/spoke2subnet",
        "name": "spoke2subnet",
        "privateEndpointNetworkPolicies": "Di

#### 1.5 Site-to-Site VPN (On-premise ↔ Hub)

**Azure Portal:**


1.  **Virtual Network Gateway 생성 (Hub VNET):**

    -   "Create a resource"에서 "Virtual network gateway"를 검색하여 선택하고 "Create"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: hub-vgw
        -   Region: Korea Central
        -   Gateway type: VPN
        -   VPN type: Route-based
        -   SKU: 적절한 SKU 선택 (예: VpnGw1)
        -   Generation: Generation1 (또는 적절한 세대 선택)
        -   Virtual network: hubvnet
        -   Gateway subnet address range: 10.0.0.0/24 (Hub VNET의 GatewaySubnet)
        -   Public IP address: 새로 생성 (예: hub-vgw-pip)
        -   Enable active-active mode: Disabled (필요에 따라 활성화)
        -   Configure BGP: Enabled
            -   ASN: 65515
    -   "Review + create"를 클릭하고 "Create"를 클릭합니다.
2.  **Local Network Gateway 생성 (Hub VNET용):**

    -   "Create a resource"에서 "Local network gateway"를 검색하여 선택하고 "Create"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: hub-lgw
        -   IP address: On-premise VPN 장치의 Public IP (VM의 Public IP)
        -   Address space: On-premise VNET 주소 공간 (192.168.0.0/16)
        -   BGP settings에서 ASN: 65001
        -   BGP peer IP address: 192.168.0.254
        -   Region: Korea Central
    -   "Review + create"를 클릭하고 "Create"를 클릭합니다.
3.  **Virtual Network Gateway 생성 (On-prem VNET):**

    -   "Create a resource"에서 "Virtual network gateway"를 검색하여 선택하고 "Create"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: onprem-vgw
        -   Region: Korea Central
        -   Gateway type: VPN
        -   VPN type: Route-based
        -   SKU: 적절한 SKU 선택 (예: VpnGw1)
        -   Generation: Generation1 (또는 적절한 세대 선택)
        -   Virtual network: onprem-vnet
        -   Gateway subnet address range: 192.168.0.0/24 (Onprem VNET의 GatewaySubnet)
        -   Public IP address: 새로 생성 (예: onprem-vgw-pip)
        -   Enable active-active mode: Disabled (필요에 따라 활성화)
        -   Configure BGP: Enabled
            - ASN: 65001
    -   "Review + create"를 클릭하고 "Create"를 클릭합니다.
4.  **Local Network Gateway 생성 (Onprem VNET용):**

    -   "Create a resource"에서 "Local network gateway"를 검색하여 선택하고 "Create"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: onprem-lgw
        -   IP address: Hub VNET VPN 장치의 Public IP 주소 (실제 주소로 변경)
        -   Address space: Hub VNET의 주소 공간 (10.0.0.0/16)
        -   Region: Korea Central
        -   BGP settings: Enabled
            -   ASN: 65515
            -   BGP peer IP address: Hub VNET GatewaySubnet 대역에서 사용하지 않는 IP (e.g., 10.0.0.254)
    -   "Review + create"를 클릭하고 "Create"를 클릭합니다.
5.  **연결 생성 (Hub VNET):**

    -   Hub VNET의 Virtual Network Gateway (hub-vgw) 페이지로 이동합니다.

-   "Connections"에서 "Add"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: hub-to-onprem
        -   Connection type: Site-to-site (IPsec)
        -   Virtual network gateway: hub-vgw
        -   Local network gateway: onprem-lgw
        -   Shared key (PSK): 공유 키 입력 (On-premise VPN 장치와 동일해야 함)
        -   Enable BGP: enabled
    -   "OK"를 클릭합니다.

6.  **연결 생성 (On-prem VNET):**
    -   Onprem VNET의 Virtual Network Gateway (onprem-vgw) 페이지로 이동합니다.
    -   "Connections"에서 "Add"를 클릭합니다.
    -   다음 정보를 입력합니다.
        -   Name: onprem-to-hub
        -   Connection type: Site-to-site (IPsec)
        -   Virtual network gateway: onprem-vgw
        -   Local network gateway: hub-lgw
        -   Shared key (PSK): 공유 키 입력 (Hub VNET VPN 장치와 동일해야 함)
        -   Enable BGP: enabled
    -   "OK"를 클릭합니다.

In [11]:
HUB_VGW_PIP_NAME="hub-vgw-pip"
SHARED_KEY="P@ssw0rd1234"
ONPREM_VGW_PIP_NAME="onprem-vgw-pip"


In [13]:
# hub vnet의 vgw용 public ip 생성

!az network public-ip create \
    --name $HUB_VGW_PIP_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Standard \
    --allocation-method Static 

{
  "publicIp": {
    "ddosSettings": {
      "protectionMode": "VirtualNetworkInherited"
    },
    "etag": "W/\"e2af700b-34e5-423c-a1dd-7b6650e5cd3e\"",
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/publicIPAddresses/hub-vgw-pip",
    "idleTimeoutInMinutes": 4,
    "ipAddress": "20.41.80.48",
    "ipTags": [],
    "location": "koreacentral",
    "name": "hub-vgw-pip",
    "provisioningState": "Succeeded",
    "publicIPAddressVersion": "IPv4",
    "publicIPAllocationMethod": "Static",
    "resourceGroup": "secu-ai-workshop",
    "resourceGuid": "fe19a418-a945-435b-8b5e-87d07c33bbcb",
    "sku": {
      "name": "Standard",
      "tier": "Regional"
    },
    "type": "Microsoft.Network/publicIPAddresses"
  }
}


In [15]:
# Hub VNET용 Public IP 주소 조회
HUB_VGW_PIP=!az network public-ip show --name $HUB_VGW_PIP_NAME --resource-group $RESOURCE_GROUP --query ipAddress --output tsv
HUB_VGW_PIP=HUB_VGW_PIP[0].strip()
!echo "Hub VNET Public IP: $HUB_VGW_PIP"

"Hub VNET Public IP: 20.41.80.48"


In [ ]:
# Hub VNET용 Virtual Network Gateway 생성
!az network vnet-gateway create \
    --name hub-vgw \
    --resource-group $RESOURCE_GROUP \
    --vnet $HUB_VNET_NAME \
    --gateway-type Vpn \
    --vpn-type RouteBased \
    --sku VpnGw1 \
    --public-ip-addresses $HUB_VGW_PIP_NAME \
    --asn 65516

In [16]:
# Hub VNET용 Local Network Gateway 생성
!az network local-gateway create \
    --name hub-lgw \
    --resource-group $RESOURCE_GROUP \
    --gateway-ip-address $HUB_VGW_PIP \
    --local-address-prefixes $HUB_ADDRESS_PREFIX \
    --asn 65516 \
    --peer-weight 0 \
    --bgp-peering-address 10.0.0.254


{
  "bgpSettings": {
    "asn": 65516,
    "bgpPeeringAddress": "10.0.0.254",
    "peerWeight": 0
  },
  "etag": "W/\"001c9098-e55d-4c5f-858a-7f9999a541c8\"",
  "gatewayIpAddress": "20.41.80.48",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/localNetworkGateways/hub-lgw",
  "localNetworkAddressSpace": {
    "addressPrefixes": [
      "10.0.0.0/16"
    ]
  },
  "location": "koreacentral",
  "name": "hub-lgw",
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "resourceGuid": "dc8b747e-07a5-423e-92ed-a7c937ae9457",
  "type": "Microsoft.Network/localNetworkGateways"
}


In [17]:
# On-prem VNET용 Public IP 주소 생성
!az network public-ip create \
        --name $ONPREM_VGW_PIP_NAME \
        --resource-group $RESOURCE_GROUP \
        --location $LOCATION \
        --allocation-method Static

{
  "publicIp": {
    "ddosSettings": {
      "protectionMode": "VirtualNetworkInherited"
    },
    "etag": "W/\"93b1cac0-333e-47ce-b6af-ece79a323eda\"",
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/publicIPAddresses/onprem-vgw-pip",
    "idleTimeoutInMinutes": 4,
    "ipAddress": "20.39.206.182",
    "ipTags": [],
    "location": "koreacentral",
    "name": "onprem-vgw-pip",
    "provisioningState": "Succeeded",
    "publicIPAddressVersion": "IPv4",
    "publicIPAllocationMethod": "Static",
    "resourceGroup": "secu-ai-workshop",
    "resourceGuid": "294eda9a-590a-4054-a4a6-702448eee5a2",
    "sku": {
      "name": "Standard",
      "tier": "Regional"
    },
    "type": "Microsoft.Network/publicIPAddresses"
  }
}


In [18]:
# On-prem VNET용 Public IP 주소 조회
ONPREM_VGW_PIP=!az network public-ip show --name $ONPREM_VGW_PIP_NAME --resource-group $RESOURCE_GROUP --query ipAddress --output tsv
ONPREM_VGW_PIP=ONPREM_VGW_PIP[0].strip()
!echo "On-prem VNET Public IP: $ONPREM_VGW_PIP"

"On-prem VNET Public IP: 20.39.206.182"


In [ ]:
# On-prem VNET용 Virtual Network Gateway 생성
!az network vnet-gateway create \
    --name onprem-vgw \
    --resource-group $RESOURCE_GROUP \
    --vnet $ONPREM_VNET_NAME \
    --gateway-type Vpn \
    --vpn-type RouteBased \
    --sku VpnGw1 \
    --public-ip-addresses $ONPREM_VGW_PIP_NAME \
    --asn 65521

In [21]:
# On-prem VNET용 Local Network Gateway 생성
!az network local-gateway create \
    --name onprem-lgw \
    --resource-group $RESOURCE_GROUP \
    --gateway-ip-address $ONPREM_VGW_PIP \
    --local-address-prefixes $ONPREM_ADDRESS_PREFIX \
    --asn 65521 \
    --peer-weight 0 \
    --bgp-peering-address 192.168.0.254

{
  "bgpSettings": {
    "asn": 65521,
    "bgpPeeringAddress": "192.168.0.254",
    "peerWeight": 0
  },
  "etag": "W/\"67f9e6b9-c352-4088-a05a-cfa65134e53f\"",
  "gatewayIpAddress": "20.39.206.182",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/localNetworkGateways/onprem-lgw",
  "localNetworkAddressSpace": {
    "addressPrefixes": [
      "192.168.0.0/16"
    ]
  },
  "location": "koreacentral",
  "name": "onprem-lgw",
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "resourceGuid": "2d0697d7-6974-41ef-92f0-ad3f2f19a353",
  "type": "Microsoft.Network/localNetworkGateways"
}


In [22]:
# Hub VNET과 On-prem VNET 간의 VPN 연결 생성
!az network vpn-connection create \
  --name hub-to-onprem \
  --resource-group $RESOURCE_GROUP \
  --vnet-gateway1 hub-vgw \
  --local-gateway2 onprem-lgw \
  --shared-key $SHARED_KEY \
  --location $LOCATION \
  --enable-bgp

{
  "resource": {
    "authenticationType": "PSK",
    "connectionMode": "Default",
    "connectionProtocol": "IKEv2",
    "connectionStatus": "Unknown",
    "connectionType": "IPsec",
    "dpdTimeoutSeconds": 0,
    "egressBytesTransferred": 0,
    "enableBgp": true,
    "ingressBytesTransferred": 0,
    "localNetworkGateway2": {
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/localNetworkGateways/onprem-lgw",
      "resourceGroup": "secu-ai-workshop"
    },
    "provisioningState": "Succeeded",
    "resourceGuid": "9acc12f6-b636-4e5d-9e90-d18142ffd648",
    "routingWeight": 10,
    "sharedKey": "P@ssw0rd1234",
    "trafficSelectorPolicies": [],
    "useLocalAzureIpAddress": false,
    "virtualNetworkGateway1": {
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworkGateways/hub-vgw",
      "resourceGroup": "secu-ai-workshop"

In [23]:
# On-prem VNET과 Hub VNET 간의 VPN 연결 생성
!az network vpn-connection create \
    --name onprem-to-hub \
    --resource-group $RESOURCE_GROUP \
    --vnet-gateway1 onprem-vgw \
    --shared-key $SHARED_KEY \
    --local-gateway2 hub-lgw \
    --location $LOCATION \
    --enable-bgp

{
  "resource": {
    "authenticationType": "PSK",
    "connectionMode": "Default",
    "connectionProtocol": "IKEv2",
    "connectionStatus": "Unknown",
    "connectionType": "IPsec",
    "dpdTimeoutSeconds": 0,
    "egressBytesTransferred": 0,
    "enableBgp": true,
    "ingressBytesTransferred": 0,
    "localNetworkGateway2": {
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/localNetworkGateways/hub-lgw",
      "resourceGroup": "secu-ai-workshop"
    },
    "provisioningState": "Succeeded",
    "resourceGuid": "88cb3be7-8b5d-47f2-af2a-c27ceac8e589",
    "routingWeight": 10,
    "sharedKey": "P@ssw0rd1234",
    "trafficSelectorPolicies": [],
    "useLocalAzureIpAddress": false,
    "virtualNetworkGateway1": {
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworkGateways/onprem-vgw",
      "resourceGroup": "secu-ai-workshop"

**Site-to-Site VPN 설명:**

Site-to-Site VPN은 On-premise 네트워크와 Azure VNET 간에 안전한 암호화된 연결을 제공합니다. Virtual Network Gateway는 VPN 연결의 엔드포인트 역할을 하며, Local Network Gateway는 상대방 네트워크(이 경우 On-premise 또는 Hub)를 나타냅니다.

#### 1.6 VNET Peering (Hub ↔ Spoke1, Spoke2)

**Azure Portal (Hub ↔ Spoke1):**

1.  Hub VNET ("hubvnet") 설정 페이지로 이동합니다.
2.  "Settings" 아래의 "Peerings"를 선택하고 "Add"를 클릭합니다.
3.  다음 정보를 입력합니다.
    -   This virtual network:

        -   Peering link name: hub-to-spoke1
        -   Traffic to remote virtual network: Allow
        -   Traffic forwarded from remote virtual network: Allow
        -   Virtual network gateway or Route Server: Use this virtual network's gateway or Route Server

    -   Remote virtual network:

        -   Peering link name: spoke1-to-hub
        -   Traffic to remote virtual network: Allow
        -   Traffic forwarded from remote virtual network: Allow
        -   Virtual network gateway or Route Server: Use the remote virtual network's gateway or Route Server
        -   Virtual network: spoke1vnet 선택

4.  "Add"를 클릭합니다.

In [24]:
# Hub VNET에서 Spoke1 VNET으로 Peering 설정
!az network vnet peering create \
    --name hub-to-spoke1 \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $HUB_VNET_NAME \
    --remote-vnet $SPOKE1_VNET_NAME \
    --allow-vnet-access \
    --allow-forwarded-traffic \
    --allow-gateway-transit

{
  "allowForwardedTraffic": true,
  "allowGatewayTransit": true,
  "allowVirtualNetworkAccess": true,
  "doNotVerifyRemoteGateways": false,
  "etag": "W/\"e28ff409-b9dd-47fe-b060-fff46aa9990e\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/hubvnet/virtualNetworkPeerings/hub-to-spoke1",
  "name": "hub-to-spoke1",
  "peerCompleteVnets": true,
  "peeringState": "Initiated",
  "peeringSyncLevel": "RemoteNotInSync",
  "provisioningState": "Succeeded",
  "remoteAddressSpace": {
    "addressPrefixes": [
      "10.1.0.0/16"
    ]
  },
  "remoteVirtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke1vnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "remoteVirtualNetworkAddressSpace": {
    "addressPrefixes": [
      "10.1.0.0/16"
    ]
  },
  "resourceGroup": "secu-ai-workshop",
  "resourceGu

In [25]:
# Spoke1 VNET에서 Hub VNET으로 Peering 설정
!az network vnet peering create \
    --name spoke1-to-hub \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $SPOKE1_VNET_NAME \
    --remote-vnet $HUB_VNET_NAME \
    --allow-vnet-access \
    --allow-forwarded-traffic \
    --use-remote-gateways

{
  "allowForwardedTraffic": true,
  "allowGatewayTransit": false,
  "allowVirtualNetworkAccess": true,
  "doNotVerifyRemoteGateways": false,
  "etag": "W/\"93990b88-99ae-4884-a46a-30b39e79c40f\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke1vnet/virtualNetworkPeerings/spoke1-to-hub",
  "name": "spoke1-to-hub",
  "peerCompleteVnets": true,
  "peeringState": "Connected",
  "peeringSyncLevel": "FullyInSync",
  "provisioningState": "Succeeded",
  "remoteAddressSpace": {
    "addressPrefixes": [
      "10.0.0.0/16"
    ]
  },
  "remoteVirtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/hubvnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "remoteVirtualNetworkAddressSpace": {
    "addressPrefixes": [
      "10.0.0.0/16"
    ]
  },
  "resourceGroup": "secu-ai-workshop",
  "resourceGuid"

**Azure Portal (Hub ↔ Spoke2) 및 AZ CLI (Hub ↔ Spoke2):**

Spoke1과 동일한 방식으로 Peering을 설정합니다. (VNET 이름만 Spoke2로 변경)

In [26]:
# Hub VNET에서 Spoke2 VNET으로 Peering 설정
!az network vnet peering create \
    --name hub-to-spoke2 \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $HUB_VNET_NAME \
    --remote-vnet $SPOKE2_VNET_NAME \
    --allow-vnet-access \
    --allow-forwarded-traffic \
    --allow-gateway-transit

{
  "allowForwardedTraffic": true,
  "allowGatewayTransit": true,
  "allowVirtualNetworkAccess": true,
  "doNotVerifyRemoteGateways": false,
  "etag": "W/\"fae6c6e7-1ece-4dec-9ec8-95fb9964a899\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/hubvnet/virtualNetworkPeerings/hub-to-spoke2",
  "name": "hub-to-spoke2",
  "peerCompleteVnets": true,
  "peeringState": "Initiated",
  "peeringSyncLevel": "RemoteNotInSync",
  "provisioningState": "Succeeded",
  "remoteAddressSpace": {
    "addressPrefixes": [
      "10.2.0.0/16"
    ]
  },
  "remoteVirtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke2vnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "remoteVirtualNetworkAddressSpace": {
    "addressPrefixes": [
      "10.2.0.0/16"
    ]
  },
  "resourceGroup": "secu-ai-workshop",
  "resourceGu

In [27]:
# Spoke2 VNET에서 Hub VNET으로 Peering 설정
!az network vnet peering create \
    --name spoke2-to-hub \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $SPOKE2_VNET_NAME \
    --remote-vnet $HUB_VNET_NAME \
    --allow-vnet-access \
    --allow-forwarded-traffic \
    --use-remote-gateways

{
  "allowForwardedTraffic": true,
  "allowGatewayTransit": false,
  "allowVirtualNetworkAccess": true,
  "doNotVerifyRemoteGateways": false,
  "etag": "W/\"6f788128-813e-4699-ab4c-b7adb250fc47\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke2vnet/virtualNetworkPeerings/spoke2-to-hub",
  "name": "spoke2-to-hub",
  "peerCompleteVnets": true,
  "peeringState": "Connected",
  "peeringSyncLevel": "FullyInSync",
  "provisioningState": "Succeeded",
  "remoteAddressSpace": {
    "addressPrefixes": [
      "10.0.0.0/16"
    ]
  },
  "remoteVirtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/hubvnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "remoteVirtualNetworkAddressSpace": {
    "addressPrefixes": [
      "10.0.0.0/16"
    ]
  },
  "resourceGroup": "secu-ai-workshop",
  "resourceGuid"

**VNET Peering 설명:**

VNET Peering을 통해 서로 다른 VNET 간에 트래픽을 라우팅할 수 있습니다. Hub VNET은 Spoke VNET의 게이트웨이 역할을 하므로 'allow-gateway-transit' 옵션을 사용하고, Spoke VNET에서는 'use-remote-gateways'를 사용하여 Hub VNET의 게이트웨이를 사용하도록 설정합니다.


#### 1.7 Azure Firewall (Hub VNET에 배치)

**Azure Portal:**

1.  "Create a resource"에서 "Firewall"을 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독을 선택합니다.
    -   Resource group: "secu-ai-workshop"을 선택합니다.
    -   Name: "hub-firewall"
    -   Region: "Korea Central"
    -   Firewall SKU: Standard (필요에 따라 선택)
    -   Firewall management: Use a Firewall Policy to manage this firewall
    -   Firewall policy: 새 정책 생성 (예: hub-firewall-policy)
    -   Choose a virtual network: Use existing: hubvnet
    -   Subnet: AzureFirewallSubnet (자동으로 선택됨)
    -   Public IP address: 새로 생성 (예: hub-firewall-pip)
3.  "Review + create"를 클릭하고 "Create"를 클릭합니다.

In [28]:
# 변수 설정
FIREWALL_NAME="hub-firewall"
FIREWALL_POLICY_NAME="hub-firewall-policy"
FIREWALL_PIP_NAME="hub-firewall-pip"

In [42]:
# extension 설치 (필요시)

!az config set extension.dynamic_install_allow_preview=true

In [43]:

!az extension add --name azure-firewall

In [29]:
# Azure Firewall Policy 생성
!az network firewall policy create \
    --name $FIREWALL_POLICY_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION

{
  "childPolicies": [],
  "etag": "a3d3564a-8965-4c53-a667-da2f70abf9db",
  "firewalls": [],
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/firewallPolicies/hub-firewall-policy",
  "location": "koreacentral",
  "name": "hub-firewall-policy",
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "ruleCollectionGroups": [],
  "sku": {
    "tier": "Standard"
  },
  "threatIntelMode": "Alert",
  "type": "Microsoft.Network/FirewallPolicies"
}


In [49]:
# Azure Firewall Public IP 생성
!az network public-ip create \
    --name $FIREWALL_PIP_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Standard

{
  "publicIp": {
    "ddosSettings": {
      "protectionMode": "VirtualNetworkInherited"
    },
    "etag": "W/\"f4b8e06a-afe3-4462-bfbd-bae2573d9bc9\"",
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/publicIPAddresses/hub-firewall-pip2",
    "idleTimeoutInMinutes": 4,
    "ipAddress": "52.231.100.241",
    "ipTags": [],
    "location": "koreacentral",
    "name": "hub-firewall-pip2",
    "provisioningState": "Succeeded",
    "publicIPAddressVersion": "IPv4",
    "publicIPAllocationMethod": "Static",
    "resourceGroup": "secu-ai-workshop",
    "resourceGuid": "d0bbfdad-4fbe-49e9-a429-9aac7c0f0861",
    "sku": {
      "name": "Standard",
      "tier": "Regional"
    },
    "type": "Microsoft.Network/publicIPAddresses"
  }
}


In [44]:
# Azure Firewall 생성
!az network firewall create \
    --name $FIREWALL_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --policy $FIREWALL_POLICY_NAME \
    --vnet-name $HUB_VNET_NAME \
    --public-ip $FIREWALL_PIP_NAME

{
  "additionalProperties": {},
  "applicationRuleCollections": [],
  "etag": "W/\"d57f241c-9963-48ba-a227-2d2daa2f8e6c\"",
  "firewallPolicy": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/firewallPolicies/hub-firewall-policy",
    "resourceGroup": "secu-ai-workshop"
  },
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/azureFirewalls/hub-firewall",
  "ipConfigurations": [],
  "location": "koreacentral",
  "name": "hub-firewall",
  "natRuleCollections": [],
  "networkRuleCollections": [],
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "sku": {
    "name": "AZFW_VNet",
    "tier": "Standard"
  },
  "threatIntelMode": "Alert",
  "type": "Microsoft.Network/azureFirewalls"
}


#### 1.8 Azure Firewall Policy (Network Rule Collection)

**Azure Portal:**

1.  생성된 Firewall Policy (hub-firewall-policy) 페이지로 이동합니다.

2.  "Settings" 아래의 "Network Rule Collections"를 선택하고 "Add network rule collection"을 클릭합니다.

3.  다음 정보를 입력하여 규칙 컬렉션을 추가합니다.

    -   **Onprem to Spoke1:**
        -   Name: onprem-to-spoke1
        -   Priority: 100
        -   Action: Allow
        -   Rules:
            -   Name: allow-all
            -   Source type: IP Address
            -   Source addresses: 192.168.0.0/16
            -   Destination type: IP Address
            -   Destination addresses: 10.1.0.0/16
            -   Protocols: Any
            -   Ports: \*
    -   **Onprem to Spoke2:**
        -   Name: onprem-to-spoke2
        -   Priority: 101
        -   Action: Allow
        -   Rules:
        -   Name: allow-all
        -   Source type: IP Address
        -   Source addresses: 192.168.0.0/16
        -   Destination type: IP Address
        -   Destination addresses: 10.2.0.0/16
        -   Protocols: Any
        -   Ports: \*
    -   **Spoke1 to Spoke2:**
        -   Name: spoke1-to-spoke2
        -   Priority: 103
        -   Action: Allow
        -   Rules:
        -   Name: allow-all
        -   Source type: IP Address
        -   Source addresses: 10.1.0.0/16
        -   Destination type: IP Address
        -   Destination addresses: 10.2.0.0/16
        -   Protocols: Any
        -   Ports: \*
    -   **Spoke2 to Spoke1:**
        -   Name: spoke2-to-spoke1
        -   Priority: 104
        -   Action: Allow
        -   Rules:
        -   Name: allow-all
        -   Source type: IP Address
        -   Source addresses: 10.2.0.0/16
        -   Destination type: IP Address
        -   Destination addresses: 10.1.0.0/16
        -   Protocols: Any
        -   Ports: \*
4.  "Add"를 클릭하여 규칙 컬렉션을 저장합니다.

In [35]:
FIREWALL_IP=!az network firewall show --name $FIREWALL_NAME --resource-group $RESOURCE_GROUP --query "ipConfigurations[0].privateIpAddress" -o tsv
#FIREWALL_IP=FIREWALL_IP[0].strip()
!echo "Azure Firewall IP: $FIREWALL_IP"

"Azure Firewall IP: []"


In [36]:
!az network firewall show \
    --name $FIREWALL_NAME \
    --resource-group $RESOURCE_GROUP

{
  "additionalProperties": {},
  "applicationRuleCollections": [],
  "etag": "W/\"ca9db86f-d185-49eb-bc55-6a773d3024f9\"",
  "firewallPolicy": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/firewallPolicies/hub-firewall-policy",
    "resourceGroup": "secu-ai-workshop"
  },
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/azureFirewalls/hub-firewall",
  "ipConfigurations": [],
  "location": "koreacentral",
  "name": "hub-firewall",
  "natRuleCollections": [],
  "networkRuleCollections": [],
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "sku": {
    "name": "AZFW_VNet",
    "tier": "Standard"
  },
  "threatIntelMode": "Alert",
  "type": "Microsoft.Network/azureFirewalls"
}


In [39]:
# Firewall Policy ID 가져오기
FIREWALL_POLICY_ID=!az network firewall policy show --name $FIREWALL_POLICY_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
FIREWALL_POLICY_ID=FIREWALL_POLICY_ID[0].strip()
!echo "Firewall Policy ID: $FIREWALL_POLICY_ID"
# Firewall IP Address
FIREWALL_IP="10.0.1.4"

"Firewall Policy ID: /subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/firewallPolicies/hub-firewall-policy"


In [40]:
# DefaultNetworkRuleCollectionGroup 생성
!az network firewall policy rule-collection-group create \
    --resource-group $RESOURCE_GROUP \
    --policy-name $FIREWALL_POLICY_NAME \
    --name DefaultNetworkRuleCollectionGroup \
    --priority 100

{
  "etag": "1520d9d9-0f44-4329-a546-f6c49dab796d",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/firewallPolicies/hub-firewall-policy/ruleCollectionGroups/DefaultNetworkRuleCollectionGroup",
  "name": "DefaultNetworkRuleCollectionGroup",
  "priority": 100,
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "ruleCollections": [],
  "type": "Microsoft.Network/FirewallPolicies/RuleCollectionGroups"
}


In [41]:
# Onprem to Spoke1 규칙 컬렉션 추가
!az network firewall policy rule-collection-group collection add-filter-collection \
    --rule-collection-group-name DefaultNetworkRuleCollectionGroup \
    --policy-name $FIREWALL_POLICY_NAME \
    --resource-group $RESOURCE_GROUP \
    --name onprem-to-spoke1 \
    --collection-priority 100 \
    --action Allow \
    --rule-type NetworkRule \
    --rule-name allow-all \
    --source-addresses=192.168.0.0/16 \
    --destination-addresses=10.1.0.0/16 \
    --ip-protocols=Any \
    --destination-ports='*'

{
  "ruleCollections": [
    {
      "action": {
        "type": "Allow"
      },
      "name": "onprem-to-spoke1",
      "priority": 100,
      "ruleCollectionType": "FirewallPolicyFilterRuleCollection",
      "rules": [
        {
          "destinationAddresses": [
            "10.1.0.0/16"
          ],
          "destinationFqdns": [],
          "destinationIpGroups": [],
          "destinationPorts": [
            "*"
          ],
          "ipProtocols": [
            "Any"
          ],
          "name": "allow-all",
          "ruleType": "NetworkRule",
          "sourceAddresses": [
            "192.168.0.0/16"
          ],
          "sourceIpGroups": []
        }
      ]
    }
  ]
}


In [ ]:

# Onprem to Spoke2 규칙 컬렉션 추가
!az network firewall policy rule-collection-group collection add-filter-collection \
    --rule-collection-group-name DefaultNetworkRuleCollectionGroup \
    --policy-name $FIREWALL_POLICY_NAME \
    --resource-group $RESOURCE_GROUP \
    --name onprem-to-spoke2 \
    --collection-priority 101 \
    --action Allow \
    --rule-type NetworkRule \
    --rule-name allow-all \
    --source-addresses 192.168.0.0/16 \
    --destination-addresses 10.2.0.0/16 \
    --ip-protocols Any \
    --destination-ports '*'

In [ ]:
# Spoke1 to Spoke2 규칙 컬렉션 추가
!az network firewall policy rule-collection-group collection add-filter-collection \
    --rule-collection-group-name DefaultNetworkRuleCollectionGroup \
    --policy-name $FIREWALL_POLICY_NAME \
    --resource-group $RESOURCE_GROUP \
    --name spoke1-to-spoke2 \
    --collection-priority 103 \
    --action Allow \
    --rule-type NetworkRule \
    --rule-name allow-all \
    --source-addresses 10.1.0.0/16 \
    --destination-addresses 10.2.0.0/16 \
    --ip-protocols Any \
    --destination-ports '*'

In [ ]:
# Spoke2 to Spoke1 규칙 컬렉션 추가
!az network firewall policy rule-collection-group collection add-filter-collection \
    --rule-collection-group-name DefaultNetworkRuleCollectionGroup \
    --policy-name $FIREWALL_POLICY_NAME \
    --resource-group $RESOURCE_GROUP \
    --name spoke2-to-spoke1 \
    --collection-priority 104 \
    --action Allow \
    --rule-type NetworkRule \
    --rule-name allow-all \
    --source-addresses 10.2.0.0/16 \
    --destination-addresses 10.1.0.0/16 \
    --ip-protocols Any \
    --destination-ports '*'

#### 1.9 Route Table

##### 1.9.1 Hub Route Table (Hub VNET GatewaySubnet에 연결)

**Azure Portal:**

1.  "Create a resource"에서 "Route table"을 검색하여 선택하고 "Create"를 클릭합니다.

2.  다음 정보를 입력합니다.

    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Location: "Korea Central"
    -   Name: "hub-route-table"
    -   Propagate gateway routes: No
3.  "Review + create"를 클릭하고 "Create"를 클릭합니다.

4.  생성된 Route Table (hub-route-table) 페이지로 이동합니다.

5.  "Settings" 아래의 "Routes"를 선택하고 "Add"를 클릭합니다.

6.  다음 정보를 입력하여 경로를 추가합니다.

    -   Route name: to-spoke1
    -   Address prefix destination: IP Addresses
    -   Destination IP addresses/CIDR ranges: 10.1.0.0/16
    -   Next hop type: Virtual appliance
    -   Next hop address: 10.0.1.4 (Azure Firewall IP)
    -   Route name: to-spoke2
    -   Address prefix destination: IP Addresses
    -   Destination IP addresses/CIDR ranges: 10.2.0.0/16
    -   Next hop type: Virtual appliance
    -   Next hop address: 10.0.1.4 (Azure Firewall IP)
7.  "Settings" 아래 "Subnets"를 선택하고 "Associate"를 클릭합니다.

    -   Virtual network: hubvnet 선택
    -   Subnet: GatewaySubnet 선택
8.  "Associate"를 클릭합니다.

In [ ]:
# Route Table 생성
!az network route-table create \
    --name hub-route-table \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --disable-bgp-route-propagation true


In [ ]:
# Spoke1으로 가는 경로 추가
!az network route-table route create \
    --name to-spoke1 \
    --resource-group $RESOURCE_GROUP \
    --route-table-name hub-route-table \
    --address-prefix 10.1.0.0/16 \
    --next-hop-type VirtualAppliance \
    --next-hop-ip-address $FIREWALL_IP


In [ ]:
# Spoke2으로 가는 경로 추가
!az network route-table route create \
    --name to-spoke2 \
    --resource-group $RESOURCE_GROUP \
    --route-table-name hub-route-table \
    --address-prefix 10.2.0.0/16 \
    --next-hop-type VirtualAppliance \
    --next-hop-ip-address $FIREWALL_IP

In [ ]:
# Hub VNET GatewaySubnet에 연결
!az network vnet subnet update \
    --name $GATEWAY_SUBNET_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $HUB_VNET_NAME \
    --route-table hub-route-table

##### 1.9.2 Spoke1 Route Table (Spoke1 VNET에 연결)

**Azure Portal:**

(Hub Route Table과 유사하게 생성하고, 경로는 0.0.0.0/0 트래픽을 Firewall로 전달)

In [ ]:
# Route Table 생성
!az network route-table create \
    --name spoke1-route-table \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --disable-bgp-route-propagation true

In [ ]:
# 0.0.0.0/0 트래픽을 Firewall로 전달하는 경로 추가
!az network route-table route create \
    --name to-firewall \
    --resource-group $RESOURCE_GROUP \
    --route-table-name spoke1-route-table \
    --address-prefix 0.0.0.0/0 \
    --next-hop-type VirtualAppliance \
    --next-hop-ip-address $FIREWALL_IP

In [ ]:
# Spoke1 VNET에 연결
!az network vnet subnet update \
    --name $SPOKE1_SUBNET_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $SPOKE1_VNET_NAME \
    --route-table spoke1-route-table

##### 1.9.3 Spoke2 Route Table (Spoke2 VNET에 연결)

**Azure Portal:**

(Spoke1 Route Table과 동일하게 생성)


In [ ]:
# Route Table 생성
!az network route-table create \
    --name spoke2-route-table \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --disable-bgp-route-propagation true

In [ ]:
# 0.0.0.0/0 트래픽을 Firewall로 전달하는 경로 추가
!az network route-table route create \
    --name to-firewall \
    --resource-group $RESOURCE_GROUP \
    --route-table-name spoke2-route-table \
    --address-prefix 0.0.0.0/0 \
    --next-hop-type VirtualAppliance \
    --next-hop-ip-address $FIREWALL_IP

In [ ]:
# Spoke2 VNET에 연결
!az network vnet subnet update \
    --name $SPOKE2_SUBNET_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $SPOKE2_VNET_NAME \
    --route-table spoke2-route-table


**Route Table 설명:**

-   **Hub Route Table:** Spoke VNET으로 향하는 트래픽을 Azure Firewall로 전달하여 보안 정책을 적용합니다.
-   **Spoke Route Table:** 인터넷으로 나가는 트래픽 (0.0.0.0/0)을 Azure Firewall로 전달하여 중앙 집중식 보안 관리를 수행합니다.